In [ ]:
! pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = 'True')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd

import re
import string
import csv
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from rouge import Rouge
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#train.csv
train_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/train.csv')

In [ ]:
#validation csv
val_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/validation.csv')

In [ ]:
#test csv
test_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/test.csv')

In [ ]:
print('train:',train_df.shape, '\nvalidation:', val_df.shape,'\ntest:', test_df.shape)

train: (287113, 3) 
validation: (13368, 3) 
test: (11490, 3)


In [ ]:
test_df.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


## Checking Null Values

In [ ]:
train_df.isna().sum()

id            0
article       0
highlights    0
dtype: int64

## Checking Duplicates

In [ ]:
train_df.duplicated(subset=['article', 'highlights']).sum()

3098

In [ ]:
train_df = train_df.drop_duplicates(subset= ['article', 'highlights'])

In [ ]:
train_df = train_df.drop(['id'], axis = 1)
train_df = train_df.reset_index(drop=True)

In [ ]:
val_df = val_df.drop(['id'], axis = 1)
val_df = val_df.reset_index(drop=True)

In [ ]:
test_df = test_df.drop(['id'], axis = 1)
test_df = test_df.reset_index(drop=True)

In [ ]:
test_df.head(1)

,article,highlights
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 60.2 MB/s 
     |████████████████████████████████| 182 kB 74.9 MB/s 


In [ ]:
import transformers

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model = "facebook/bart-large-cnn", truncation=True)

In [ ]:
test_df.article[1]

"A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. Rahul Kumar, 17, clambered over the enclosure fence at the\xa0Kamla Nehru Zoological Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security. The intoxicated teenager ran towards the lions, shouting: 'Today I kill a lion or a lion kills me!' A zoo spokesman said: 'Guards had earlier spotted him close to the enclosure but had no idea he was planing to enter it. 'Fortunately, there are eight moats to cross before getting to where the lions usually are an

In [ ]:
test_df.article[0]

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for sp

In [ ]:
len(test_df)

11490

In [ ]:
hyps = []
with open('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_2.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Article', 'Original Summary', 'Model Output'])
    for i in range(500):
        our_summ = summarizer(test_df.article[i])
        print(our_summ)
        hyps.append(our_summ)
        writer.writerow([test_df.article[i], test_df.highlights[i], our_summ])

[{'summary_text': "U.S consumer advisory group set up by the Department of Transportation said that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. Tests conducted by the FAA use planes with a 31 inch pitch, a standard which on some airlines has decreased."}]


Your max_length is set to 142, but you input_length is only 133. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)


[{'summary_text': "Rahul Kumar, 17, clambered over enclosure fence at\xa0Kamla Nehru Zoological Park. He ran towards the animals shouting 'Today I kill a lion or a lion kills me!' Fortunately, Mr Kumar fell into a moat and was rescued by zoo security. He has been cautioned and will be sent for psychiatric evaluation."}]
[{'summary_text': 'Dougie Freedman is set to sign a new two-year deal at Nottingham Forest. Freedman has stabilised Forest since replacing Stuart Pearce in February. Forest made an audacious attempt on the play-off places when Freedman replaced Pearce but have tailed off in recent weeks.'}]
[{'summary_text': 'Fiorentina goalkeeper Neto is wanted by a number of top European clubs, according to his agent Stefano Castagna. Liverpool were linked with a move for the 25-year-old earlier in the season when Simon Mignolet was dropped from the side. A January move for Neto never materialised but the former Atletico Paranaense keeper looks certain to leave the Florence-based club

KeyboardInterrupt: ignored

In [ ]:
result_2_596 = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_2.csv')

In [ ]:
result_2_596.head() 

,Article,Original Summary,Model Output
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,"[{'summary_text': ""U.S consumer advisory group..."
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,"[{'summary_text': ""Rahul Kumar, 17, clambered ..."
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,[{'summary_text': 'Dougie Freedman is set to s...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,[{'summary_text': 'Fiorentina goalkeeper Neto ...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...","[{'summary_text': ""The former Olympian and rea..."


In [ ]:
result_2_596.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Article           595 non-null    object
 1   Original Summary  595 non-null    object
 2   Model Output      595 non-null    object
dtypes: object(3)
memory usage: 14.1+ KB


In [ ]:
result_2_596['Model Output']=result_2_596['Model Output'][17:]

In [ ]:
for i in range(len(result_2_596)):
  result_2_596['Model Output'][i] = result_2_596['Model Output'][i][19:-3]
result_2_596.head()

,Article,Original Summary,Model Output
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,U.S consumer advisory group set up by the Depa...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,"Rahul Kumar, 17, clambered over enclosure fenc..."
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman is set to sign a new two-year ...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Fiorentina goalkeeper Neto is wanted by a numb...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...","The former Olympian and reality TV star, 65, w..."


In [ ]:
rouge = Rouge()
rouge.get_scores(result_2_596['Original Summary'], result_2_596['Model Output'], avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.3671809315694561,
  'p': 0.4505652235323197,
  'f': 0.39709139791616616},
 'rouge-2': {'r': 0.16698939460587375,
  'p': 0.2090668148735699,
  'f': 0.18109517999339936},
 'rouge-l': {'r': 0.3454183846545664,
  'p': 0.42283323328128053,
  'f': 0.373167089903442}}